In [1]:
import os
import time
import torch
import sys
import torch.optim as optim
import itertools
from torchvision import transforms as T
%matplotlib notebook

In [2]:
CWD             = os.getcwd()
DATASET_DIR     = os.path.join(CWD,'data/nyuv2')
print(DATASET_DIR)

C:\Users\tomav\Documents\GitHub\cs236781-project\data/nyuv2


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [ ]:
hyperparameters_filename = "hyperparameters.py"

IMAGE_SIZES             = [(224,224), (64, 64)] #(448, 448)
TRAIN_TEST_RATIOS       = [0.5] # [0.9]
BATCH_SIZES             = [4]
NUM_WORKERSES           = [4]

BETASES                 = [(0.9,0.99)]
LRS                     = [0.001]
MOMENTUMS               = [0.9]
WEIGHT_DECAYS           = [0.0005]

STEP_SIZES              = [1000]
GAMMAS                  = [0.1]

NUM_EPOCHSES            = [1000]

all_combintations = list(itertools.product(*[IMAGE_SIZES, TRAIN_TEST_RATIOS, BATCH_SIZES, NUM_WORKERSES, \
                                             BETASES, LRS, MOMENTUMS, WEIGHT_DECAYS, STEP_SIZES, GAMMAS, \
                                             NUM_EPOCHSES]))

for combintation in all_combintations:
    # Create the current hyper-parameters file
    IMAGE_SIZE       = combintation[0]
    TRAIN_TEST_RATIO = combintation[1]
    BATCH_SIZE       = combintation[2]
    NUM_WORKERS      = combintation[3]

    BETAS            = combintation[4]
    LR               = combintation[5]
    MOMENTUM         = combintation[6]
    WEIGHT_DECAY     = combintation[7]

    STEP_SIZE        = combintation[8]
    GAMMA            = combintation[9]

    NUM_EPOCHS       = combintation[10]

    with open(hyperparameters_filename, "w") as hyperparameters_file:
        print(f"IMAGE_SIZE={IMAGE_SIZE}", file=hyperparameters_file)
        print(f"TRAIN_TEST_RATIO={TRAIN_TEST_RATIO}", file=hyperparameters_file)
        print(f"BATCH_SIZE={BATCH_SIZE}", file=hyperparameters_file)
        print(f"NUM_WORKERS={NUM_WORKERS}", file=hyperparameters_file)
        print(f"", file=hyperparameters_file)
        print(f"BETAS={BETAS}", file=hyperparameters_file)
        print(f"LR={LR}", file=hyperparameters_file)
        print(f"MOMENTUM={MOMENTUM}", file=hyperparameters_file)
        print(f"WEIGHT_DECAY={WEIGHT_DECAY}", file=hyperparameters_file)
        print(f"", file=hyperparameters_file)
        print(f"STEP_SIZE={STEP_SIZE}", file=hyperparameters_file)
        print(f"GAMMA={GAMMA}", file=hyperparameters_file)
        print(f"", file=hyperparameters_file)
        print(f"NUM_EPOCHS={NUM_EPOCHS}", file=hyperparameters_file)

    from models import SpecialFuseNetModel
    from data_manager import rgbd_gradients_dataset, rgbd_gradients_dataloader
    from train import FuseNetTrainer

    print("Current Hyper-Parameters:")
    with open(hyperparameters_filename, "r") as hyperparameters_file:
        print(hyperparameters_file.read())
    
    rgbd_grads_ds = rgbd_gradients_dataset(root=DATASET_DIR, use_transforms=True)

    dl_train,dl_test = rgbd_gradients_dataloader(root=DATASET_DIR, use_transforms=True)

    # _ = plot.rgbd_gradients_dataset_first_n(dataset=rgbd_grads_ds,n=5)
    print(f'Found {len(rgbd_grads_ds)} images in dataset folder.')

    sample_batch = next(iter(dl_train))
    rgb_size = tuple(sample_batch['rgb'].shape[1:])
    depth_size = tuple(sample_batch['depth'].shape[1:])
    grads_size = tuple(sample_batch['x'].shape[1:])

    # Train
    fusenetmodel = SpecialFuseNetModel(rgb_size=rgb_size,depth_size=depth_size,grads_size=grads_size, device=device)
    trainer = FuseNetTrainer(model=fusenetmodel, device=device)
    checkpoint_file = 'checkpoints/special_fusenet'
    if os.path.isfile(f'{checkpoint_file}.pt'):
        os.remove(f'{checkpoint_file}.pt')
    res = trainer.fit(dl_train, dl_test, early_stopping=1000, print_every=10, checkpoints=checkpoint_file)

Current Hyper-Parameters:
IMAGE_SIZE=(224, 224)
TRAIN_TEST_RATIO=0.5
BATCH_SIZE=4
NUM_WORKERS=4

BETAS=(0.9, 0.99)
LR=0.001
MOMENTUM=0.9
WEIGHT_DECAY=0.0005

STEP_SIZE=1000
GAMMA=0.1

NUM_EPOCHS=1000

Found 6 images in dataset folder.
[I] - default optimizer set: SGD(lr=0.001,momentum=0.9,weight_decay=0.0005)
[I] - default scheduler set: StepSR(step_size=1000,gamma=0.1)
--- EPOCH 1/1000 ---
test_batch (Avg. Loss 0.014: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.14s/it]
[info] - Saved checkpoint checkpoints/special_fusenet.pt at epoch 2
test_batch (Avg. Loss 0.014: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.09s/it]
[info] - Saved checkpoint checkpoints/special_fusenet.pt at epoch 3
test_batch:   0%|                                                                                | 0/1 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x0000014F7B549C18>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\torch\utils\data\dataloader.py", line 961, in __del__
    self._shutdown_workers()
  File "C:\ProgramData\Anaconda3\lib\site-packages\torch\utils\data\dataloader.py", line 941, in _shutdown_workers
    w.join()
  File "C:\ProgramData\Anaconda3\lib\multiprocessing\process.py", line 140, in join
    res = self._popen.wait(timeout)
  File "C:\ProgramData\Anaconda3\lib\multiprocessing\popen_spawn_win32.py", line 104, in wait
    res = _winapi.WaitForSingleObject(int(self._handle), msecs)
KeyboardInterrupt: 


test_batch (Avg. Loss 0.014: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.14s/it]
[info] - Saved checkpoint checkpoints/special_fusenet.pt at epoch 4
test_batch (Avg. Loss 0.014: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.10s/it]
[info] - Saved checkpoint checkpoints/special_fusenet.pt at epoch 5
test_batch (Avg. Loss 0.014: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.13s/it]
[info] - Saved checkpoint checkpoints/special_fusenet.pt at epoch 6
test_batch (Avg. Loss 0.014: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.13s/it]
[info] - Saved checkpoint checkpoints/special_fusenet.pt at epoch 7
test_batch (Avg. Loss 0.014: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.11s/it]
[info] - Saved checkpoint checkpoints/special_fusenet.pt at epoch 8
test_batch (Avg. Loss 0.014: 100%|██████████████████████████

test_batch (Avg. Loss 0.013: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.13s/it]
[info] - Saved checkpoint checkpoints/special_fusenet.pt at epoch 31
test_batch (Avg. Loss 0.013: 100%|███████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.14s/it]
[info] - Saved checkpoint checkpoints/special_fusenet.pt at epoch 32
test_batch:   0%|                                                                                | 0/1 [00:00<?, ?it/s]

In [ ]:
# fusenetmodel = SpecialFuseNetModel(rgb_size=rgb_size,depth_size=depth_size,grads_size=grads_size, device=device)

In [ ]:
# trainer = FuseNetTrainer(model=fusenetmodel, device=device)

In [ ]:
# checkpoint_file = 'checkpoints/special_fusenet'
# if os.path.isfile(f'{checkpoint_file}.pt'):
#     os.remove(f'{checkpoint_file}.pt')

In [ ]:
# res = trainer.fit(dl_train, dl_test, early_stopping=1000, print_every=10, checkpoints=checkpoint_file)